# Деревья

![](src/1.png)

1. Нечто, действительно имеющее древовидную форму — деревья каталогов на дисках, деревья сцен в 3D, деревья принятия решений.
2. Деревья поиска — структуры данных, позволяющие добавлять-убирать объекты и позволяющие быстрый поиск по ключу. Например, словари, индексы БД.
3. Куча — используется как вспомогательная в алгоритмах.
4. Двоичное разбиение пространства в 3D — известный способ сортировки от дальних к ближним.

## Пример построения дерева файлов из файловой системы

![](src/2.png)

![](src/3.png)

In [1]:
import os
import pathlib

PIPE = "│"
ELBOW = "└──"
TEE = "├──"
PIPE_PREFIX = "│   "
SPACE_PREFIX = "    "

In [5]:
class DirectoryTree:
    def __init__(self, root_dir):
        self._generator = _TreeGenerator(root_dir)

    def generate(self):
        tree = self._generator.build_tree()
        for entry in tree:
            print(entry)

In [6]:
class _TreeGenerator:
    def __init__(self, root_dir):
        self._root_dir = pathlib.Path(root_dir)
        self._tree = []

    def build_tree(self):
        self._tree_head()
        self._tree_body(self._root_dir)
        return self._tree

    def _tree_head(self):
        self._tree.append(f"{self._root_dir}{os.sep}")
        self._tree.append(PIPE)

    def _tree_body(self, directory, prefix=""):
        entries = directory.iterdir()
        entries = sorted(entries, key=lambda entry: entry.is_file())
        entries_count = len(entries)
        for index, entry in enumerate(entries):
            connector = ELBOW if index == entries_count - 1 else TEE
            if entry.is_dir():
                self._add_directory(
                    entry, index, entries_count, prefix, connector
                )
            else:
                self._add_file(entry, prefix, connector)

    def _add_directory(
        self, directory, index, entries_count, prefix, connector
    ):
        self._tree.append(f"{prefix}{connector} {directory.name}{os.sep}")
        if index != entries_count - 1:
            prefix += PIPE_PREFIX
        else:
            prefix += SPACE_PREFIX
        self._tree_body(
            directory=directory,
            prefix=prefix,
        )
        self._tree.append(prefix.rstrip())

    def _add_file(self, file, prefix, connector):
        self._tree.append(f"{prefix}{connector} {file.name}")

In [7]:
tree = DirectoryTree("../11lect")

In [8]:
tree.generate()

../11lect/
│
├── docker-rabbit/
│   ├── rabbit_files/
│   │   ├── mnesia/
│   │   │   ├── rabbit@rabbitmq/
│   │   │   │   ├── coordination/
│   │   │   │   │   └── rabbit@rabbitmq/
│   │   │   │   │       ├── 00000002.wal
│   │   │   │   │       ├── meta.dets
│   │   │   │   │       └── names.dets
│   │   │   │   │
│   │   │   │   │
│   │   │   │   ├── msg_stores/
│   │   │   │   │   └── vhosts/
│   │   │   │   │       └── 628WB79CIFDYO9LJI6DKMI09L/
│   │   │   │   │           ├── msg_store_transient/
│   │   │   │   │           │   ├── file_summary.ets
│   │   │   │   │           │   ├── clean.dot
│   │   │   │   │           │   ├── msg_store_index.ets
│   │   │   │   │           │   └── 0.rdq
│   │   │   │   │           │
│   │   │   │   │           ├── msg_store_persistent/
│   │   │   │   │           │   ├── file_summary.ets
│   │   │   │   │           │   ├── clean.dot
│   │   │   │   │           │   ├── msg_store_index.ets
│   │   │   │   │           │   └── 0.rdq
│   │   │   │ 

### Если хочется опцию для показа только директорий

In [9]:
class _TreeGenerator:
    def __init__(self, root_dir, dir_only=False):
        self._root_dir = pathlib.Path(root_dir)
        self._dir_only = dir_only
        self._tree = []

    def _tree_body(self, directory, prefix=""):
        entries = self._prepare_entries(directory)
        entries_count = len(entries)
        for index, entry in enumerate(entries):
            connector = ELBOW if index == entries_count - 1 else TEE
            if entry.is_dir():
                self._add_directory(
                    entry, index, entries_count, prefix, connector
                )
            else:
                self._add_file(entry, prefix, connector)

    def _prepare_entries(self, directory):
        entries = directory.iterdir()
        if self._dir_only:
            entries = [entry for entry in entries if entry.is_dir()]
            return entries
        entries = sorted(entries, key=lambda entry: entry.is_file())
        return entries

    def build_tree(self):
        self._tree_head()
        self._tree_body(self._root_dir)
        return self._tree

    def _tree_head(self):
        self._tree.append(f"{self._root_dir}{os.sep}")
        self._tree.append(PIPE)

    def _add_directory(
        self, directory, index, entries_count, prefix, connector
    ):
        self._tree.append(f"{prefix}{connector} {directory.name}{os.sep}")
        if index != entries_count - 1:
            prefix += PIPE_PREFIX
        else:
            prefix += SPACE_PREFIX
        self._tree_body(
            directory=directory,
            prefix=prefix,
        )
        self._tree.append(prefix.rstrip())

    def _add_file(self, file, prefix, connector):
        self._tree.append(f"{prefix}{connector} {file.name}")



In [10]:
class DirectoryTree:
    def __init__(self, root_dir, dir_only=False):
        self._generator = _TreeGenerator(root_dir, dir_only)

    def generate(self):
        tree = self._generator.build_tree()
        for entry in tree:
            print(entry)

In [11]:
tree = DirectoryTree('../11lect', dir_only=True)
tree.generate()

../11lect/
│
├── docker-rabbit/
│   ├── rabbit_files/
│   │   └── mnesia/
│   │       ├── rabbit@rabbitmq/
│   │       │   ├── coordination/
│   │       │   │   └── rabbit@rabbitmq/
│   │       │   │
│   │       │   │
│   │       │   ├── msg_stores/
│   │       │   │   └── vhosts/
│   │       │   │       └── 628WB79CIFDYO9LJI6DKMI09L/
│   │       │   │           ├── msg_store_transient/
│   │       │   │           │
│   │       │   │           ├── msg_store_persistent/
│   │       │   │           │
│   │       │   │           └── queues/
│   │       │   │               └── 7H5R02C8OIY2VA2HNBTO2ZGLS/
│   │       │   │
│   │       │   │
│   │       │   │
│   │       │   │
│   │       │   │
│   │       │   └── quorum/
│   │       │       └── rabbit@rabbitmq/
│   │       │
│   │       │
│   │       │
│   │       └── rabbit@rabbitmq-plugins-expand/
│   │
│   │
│   │
│   └── .ipynb_checkpoints/
│
│
├── .ipynb_checkpoints/
│
└── src/



In [12]:
tree = DirectoryTree('../11lect', dir_only=False)
tree.generate()

../11lect/
│
├── docker-rabbit/
│   ├── rabbit_files/
│   │   ├── mnesia/
│   │   │   ├── rabbit@rabbitmq/
│   │   │   │   ├── coordination/
│   │   │   │   │   └── rabbit@rabbitmq/
│   │   │   │   │       ├── 00000002.wal
│   │   │   │   │       ├── meta.dets
│   │   │   │   │       └── names.dets
│   │   │   │   │
│   │   │   │   │
│   │   │   │   ├── msg_stores/
│   │   │   │   │   └── vhosts/
│   │   │   │   │       └── 628WB79CIFDYO9LJI6DKMI09L/
│   │   │   │   │           ├── msg_store_transient/
│   │   │   │   │           │   ├── file_summary.ets
│   │   │   │   │           │   ├── clean.dot
│   │   │   │   │           │   ├── msg_store_index.ets
│   │   │   │   │           │   └── 0.rdq
│   │   │   │   │           │
│   │   │   │   │           ├── msg_store_persistent/
│   │   │   │   │           │   ├── file_summary.ets
│   │   │   │   │           │   ├── clean.dot
│   │   │   │   │           │   ├── msg_store_index.ets
│   │   │   │   │           │   └── 0.rdq
│   │   │   │ 

## Создание дерева из словаря

In [19]:
import pprint

def list_to_tree(data):
    out = { 
        0: { 'id': 0, 'parent_id': 0, 'name': "Root node", 'sub': [] }
    }


#     pprint.pprint(out)
#     print()
    for p in data:
        # добавляем в словарь родительский элемент
        out.setdefault(p['parent_id'], { 'sub': [] })
        
        # добавляем в словарь элемент, под id p
        out.setdefault(p['id'], { 'sub': [] })
        
        # добавляем информацию о p в элемент под ключом p в словаре
        out[p['id']].update(p)
        
        # добавляем дочерние элементы в родительский
        out[p['parent_id']]['sub'].append(out[p['id']])
        
        # вывод информации
#         pprint.pprint(out)
#         print()

    return out[0]

In [20]:
import pprint

data = [
    { 'id': 1, 'parent_id': 2, 'name': "Node1" },
    { 'id': 2, 'parent_id': 5, 'name': "Node2" },
    { 'id': 3, 'parent_id': 0, 'name': "Node3" },
    { 'id': 4, 'parent_id': 5, 'name': "Node4" },
    { 'id': 5, 'parent_id': 0, 'name': "Node5" },
    { 'id': 6, 'parent_id': 3, 'name': "Node6" },
    { 'id': 7, 'parent_id': 3, 'name': "Node7" },
    { 'id': 8, 'parent_id': 0, 'name': "Node8" },
    { 'id': 9, 'parent_id': 1, 'name': "Node9" }
]
pprint.pprint(list_to_tree(data))

{'id': 0,
 'name': 'Root node',
 'parent_id': 0,
 'sub': [{'id': 3,
          'name': 'Node3',
          'parent_id': 0,
          'sub': [{'id': 6, 'name': 'Node6', 'parent_id': 3, 'sub': []},
                  {'id': 7, 'name': 'Node7', 'parent_id': 3, 'sub': []}]},
         {'id': 5,
          'name': 'Node5',
          'parent_id': 0,
          'sub': [{'id': 2,
                   'name': 'Node2',
                   'parent_id': 5,
                   'sub': [{'id': 1,
                            'name': 'Node1',
                            'parent_id': 2,
                            'sub': [{'id': 9,
                                     'name': 'Node9',
                                     'parent_id': 1,
                                     'sub': []}]}]},
                  {'id': 4, 'name': 'Node4', 'parent_id': 5, 'sub': []}]},
         {'id': 8, 'name': 'Node8', 'parent_id': 0, 'sub': []}]}


## XML файлы

![](src/5.png)

```
<bookstore>
   <book category="COOKING">
      <title lang="en">Everyday Italian</title>
      <author>Giada De Laurentiis</author>
      <year>2005</year>
      <price>30.00</price>
   </book>
   <book category="CHILDREN">
      <title lang="en">Harry Potter</title>
      <author>J K. Rowling</author>
      <year>2005</year>
      <price>29.99</price>
   </book>
   <book category="WEB">
      <title lang="en">Learning XML</title>
      <author>Erik T. Ray</author>
      <year>2003</year>
      <price>39.95</price>
   </book>
</bookstore>

```

### Еще пример
![](src/4.jpeg)

In [21]:
from xml.etree import ElementTree

def indent(elem, level=0):
    i = "\n" + level*"  "
    j = "\n" + (level-1)*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for subelem in elem:
            indent(subelem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = j
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = j
    return elem        

In [22]:
root = ElementTree.parse('src/file2.xml').getroot()
indent(root)
ElementTree.dump(root)

<note>
  <to>Tove</to>
<from>Jani</from>
<heading>Reminder</heading>
<body>Don't forget me this weekend!</body>
</note>


In [23]:
root = ElementTree.parse('src/file1.xml').getroot()
indent(root)
ElementTree.dump(root)

<breakfast_menu>
  <food>
    <name>Belgian Waffles</name>
  <price>$5.95</price>
  <description>
   Two of our famous Belgian Waffles with plenty of real maple syrup
   </description>
  <calories>650</calories>
  </food>
<food>
    <name>Strawberry Belgian Waffles</name>
  <price>$7.95</price>
  <description>
    Light Belgian waffles covered with strawberries and whipped cream
    </description>
  <calories>900</calories>
  </food>
<food>
    <name>Berry-Berry Belgian Waffles</name>
  <price>$8.95</price>
  <description>
    Belgian waffles covered with assorted fresh berries and whipped cream
    </description>
  <calories>900</calories>
  </food>
<food>
    <name>French Toast</name>
  <price>$4.50</price>
  <description>
    Thick slices made from our homemade sourdough bread
    </description>
  <calories>600</calories>
  </food>
<food>
    <name>Homestyle Breakfast</name>
  <price>$6.95</price>
  <description>
    Two eggs, bacon or sausage, toast, and our ever-popular hash brown

## Индексы в базах данных

**Индексы в PostgreSQL** — специальные объекты базы данных, предназначенные в основном для ускорения доступа к данным.

**Строки идентифицируются с помощью TID (tuple id)**, который состоит из номера блока файла и позиции строки внутри блока. Тогда, зная ключ или некоторую информацию о нем, можно быстро прочитать те строки, в которых может находиться интересующая нас информация, не просматривая всю таблицу полностью.

![](src/6.png)

### Поиск по равенству
![](src/7.png)

### Поиск по неравенству
![](src/8.png)

### Поиск по диапазону
![](src/9.png)